# Module 11: Final Project - Complete Feature Engineering Pipeline

**Difficulty**: ⭐⭐⭐ Advanced  
**Estimated Time**: 90 minutes  
**Prerequisites**: Modules 00-10 (All previous modules)

## Learning Objectives

By the end of this notebook, you will be able to:

1. Build an end-to-end feature engineering pipeline using sklearn Pipeline and ColumnTransformer
2. Handle messy real-world data with missing values, mixed types, and outliers
3. Combine all techniques learned: encoding, scaling, feature creation, selection
4. Process different column types appropriately (numerical, categorical, datetime, text)
5. Create a production-ready, reusable pipeline
6. Demonstrate dramatic improvement from raw data to fully engineered features

## 1. Project Overview

**Goal**: Build a complete feature engineering pipeline for predicting e-commerce order value.

**Dataset characteristics** (realistic messy data!):
- Mixed data types (numerical, categorical, datetime, text)
- Missing values
- Outliers
- Skewed distributions
- Category imbalances

**We'll apply ALL techniques from Modules 00-10**:
1. Handle missing data (Module 01)
2. Encode categorical variables (Module 02)
3. Scale numerical features (Module 03)
4. Create polynomial features and interactions (Module 04)
5. Bin continuous variables (Module 05)
6. Extract datetime features (Module 06)
7. Vectorize text data (Module 07)
8. Select important features (Module 08)
9. Interpret results (Module 09)
10. Automate where appropriate (Module 10)

## 2. Setup

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Feature engineering
from sklearn.preprocessing import (
    StandardScaler, RobustScaler, OneHotEncoder, OrdinalEncoder,
    FunctionTransformer, KBinsDiscretizer
)
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Configuration
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set random seed for reproducibility
np.random.seed(42)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.precision', 2)

print("✓ Setup complete!")

## 3. Create Realistic Messy Dataset

Simulate a real e-commerce orders dataset with all the challenges you'd face in practice.

In [ ]:
# Generate realistic e-commerce order data
n_orders = 2000
np.random.seed(42)

# Generate order dates (last 2 years)
start_date = pd.Timestamp('2022-01-01')
order_dates = [start_date + timedelta(days=np.random.randint(0, 730)) for _ in range(n_orders)]

# Customer data
customer_ages = np.random.randint(18, 75, n_orders)
customer_types = np.random.choice(['New', 'Returning', 'VIP'], n_orders, p=[0.3, 0.6, 0.1])
account_ages_days = np.random.randint(1, 1000, n_orders)

# Product data
categories = np.random.choice(
    ['Electronics', 'Clothing', 'Home', 'Books', 'Sports', 'Beauty'],
    n_orders,
    p=[0.25, 0.25, 0.15, 0.15, 0.10, 0.10]
)
num_items = np.random.poisson(lam=2, size=n_orders) + 1  # At least 1 item
item_prices = np.random.gamma(shape=2, scale=30, size=n_orders)  # Skewed distribution

# Shipping data
shipping_methods = np.random.choice(
    ['Standard', 'Express', 'Overnight'],
    n_orders,
    p=[0.7, 0.2, 0.1]
)
countries = np.random.choice(
    ['USA', 'UK', 'Canada', 'Australia', 'Germany'],
    n_orders,
    p=[0.5, 0.2, 0.15, 0.1, 0.05]
)

# Review text (simplified)
positive_words = ['great', 'excellent', 'love', 'perfect', 'amazing', 'recommend']
negative_words = ['poor', 'disappointing', 'bad', 'waste', 'terrible', 'defective']
neutral_words = ['okay', 'average', 'fine', 'decent', 'acceptable']

reviews = []
for _ in range(n_orders):
    sentiment = np.random.choice(['positive', 'negative', 'neutral'], p=[0.6, 0.2, 0.2])
    if sentiment == 'positive':
        words = np.random.choice(positive_words, 3)
    elif sentiment == 'negative':
        words = np.random.choice(negative_words, 3)
    else:
        words = np.random.choice(neutral_words, 3)
    reviews.append(' '.join(words))

# Create base dataframe
df = pd.DataFrame({
    'order_date': order_dates,
    'customer_age': customer_ages,
    'customer_type': customer_types,
    'account_age_days': account_ages_days,
    'product_category': categories,
    'num_items': num_items,
    'avg_item_price': item_prices,
    'shipping_method': shipping_methods,
    'country': countries,
    'review_text': reviews
})

# Calculate target (order value) with realistic patterns
base_value = df['num_items'] * df['avg_item_price']
category_multiplier = df['product_category'].map({
    'Electronics': 1.5, 'Clothing': 0.8, 'Home': 1.2,
    'Books': 0.6, 'Sports': 1.0, 'Beauty': 0.9
})
customer_multiplier = df['customer_type'].map({
    'New': 0.8, 'Returning': 1.0, 'VIP': 1.5
})
shipping_fee = df['shipping_method'].map({
    'Standard': 5, 'Express': 15, 'Overnight': 30
})

df['order_value'] = (
    base_value * category_multiplier * customer_multiplier +
    shipping_fee +
    np.random.normal(0, 20, n_orders)  # Random noise
)

# Add realistic messiness!

# 1. Missing values
missing_mask_age = np.random.rand(n_orders) < 0.10
df.loc[missing_mask_age, 'customer_age'] = np.nan

missing_mask_review = np.random.rand(n_orders) < 0.15
df.loc[missing_mask_review, 'review_text'] = np.nan

# 2. Outliers
outlier_mask = np.random.rand(n_orders) < 0.02
df.loc[outlier_mask, 'order_value'] *= 5  # Some very large orders

# 3. Data quality issues
df.loc[df.sample(5).index, 'customer_type'] = None  # Missing category

print(f"Created e-commerce dataset: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nTarget: order_value")
print(f"  Mean: ${df['order_value'].mean():.2f}")
print(f"  Median: ${df['order_value'].median():.2f}")
print(f"  Min: ${df['order_value'].min():.2f}")
print(f"  Max: ${df['order_value'].max():.2f}")
print(f"\nDataset info:")
df.info()

In [ ]:
# Display sample data
print("Sample orders:")
df.head(10)

In [ ]:
# Check data quality issues
print("Data Quality Report:")
print("="*60)

print("\n1. Missing Values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(1)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

print("\n2. Data Types:")
print(df.dtypes)

print("\n3. Numerical Distributions:")
print(df[['customer_age', 'num_items', 'avg_item_price', 'order_value']].describe())

print("\n4. Categorical Distributions:")
for col in ['customer_type', 'product_category', 'shipping_method']:
    print(f"\n{col}:")
    print(df[col].value_counts())

In [ ]:
# Visualize distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Order value distribution
axes[0, 0].hist(df['order_value'], bins=50, edgecolor='black')
axes[0, 0].set_xlabel('Order Value ($)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Order Value Distribution (Notice outliers!)', fontweight='bold')

# Category distribution
df['product_category'].value_counts().plot(kind='barh', ax=axes[0, 1], edgecolor='black')
axes[0, 1].set_xlabel('Count')
axes[0, 1].set_title('Product Category Distribution', fontweight='bold')

# Customer age distribution
axes[1, 0].hist(df['customer_age'].dropna(), bins=30, edgecolor='black')
axes[1, 0].set_xlabel('Customer Age')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Customer Age Distribution (with missing values)', fontweight='bold')

# Order value by category
df.boxplot(column='order_value', by='product_category', ax=axes[1, 1])
axes[1, 1].set_xlabel('Product Category')
axes[1, 1].set_ylabel('Order Value ($)')
axes[1, 1].set_title('Order Value by Category', fontweight='bold')
plt.suptitle('')  # Remove default title

plt.tight_layout()
plt.show()

## 4. Split Data First!

**CRITICAL**: Always split before any feature engineering to avoid data leakage.

In [ ]:
# Separate features and target
X = df.drop('order_value', axis=1)
y = df['order_value']

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\n✓ Data split complete - NOW we can engineer features!")

## 5. Baseline Model (No Feature Engineering)

Always start with a baseline to measure improvement!

In [ ]:
# For baseline, use only numerical features that are complete
baseline_features = ['num_items', 'account_age_days']

X_train_baseline = X_train[baseline_features].fillna(0)
X_test_baseline = X_test[baseline_features].fillna(0)

# Train simple model
baseline_model = Ridge(alpha=1.0)
baseline_model.fit(X_train_baseline, y_train)

# Evaluate
y_pred_baseline = baseline_model.predict(X_test_baseline)
rmse_baseline = np.sqrt(mean_squared_error(y_test, y_pred_baseline))
mae_baseline = mean_absolute_error(y_test, y_pred_baseline)
r2_baseline = r2_score(y_test, y_pred_baseline)

print("BASELINE PERFORMANCE (minimal features, no engineering):")
print("="*60)
print(f"Features used: {baseline_features}")
print(f"RMSE: ${rmse_baseline:.2f}")
print(f"MAE: ${mae_baseline:.2f}")
print(f"R² Score: {r2_baseline:.3f}")
print("="*60)
print("\nGoal: Beat this with feature engineering!")

## 6. Build Feature Engineering Pipeline

Now let's build a comprehensive pipeline using **ColumnTransformer** and **Pipeline**.

**Strategy**:
- Numerical features: Impute → Scale → Create interactions
- Categorical features: Impute → One-hot encode
- Datetime features: Extract components → Create cyclical features
- Text features: Impute → TF-IDF vectorize

In [ ]:
# Define column types
numerical_features = ['customer_age', 'account_age_days', 'num_items', 'avg_item_price']
categorical_features = ['customer_type', 'product_category', 'shipping_method', 'country']
datetime_features = ['order_date']
text_features = ['review_text']

print("Feature types:")
print(f"  Numerical: {numerical_features}")
print(f"  Categorical: {categorical_features}")
print(f"  Datetime: {datetime_features}")
print(f"  Text: {text_features}")

In [ ]:
# Custom transformer for datetime features
from sklearn.base import BaseEstimator, TransformerMixin

class DatetimeFeatureExtractor(BaseEstimator, TransformerMixin):
    """
    Extract datetime features: month, day of week, quarter, cyclical encodings.
    """
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        
        # Ensure datetime type
        X['order_date'] = pd.to_datetime(X['order_date'])
        
        # Extract components
        features = pd.DataFrame()
        features['month'] = X['order_date'].dt.month
        features['day_of_week'] = X['order_date'].dt.dayofweek
        features['quarter'] = X['order_date'].dt.quarter
        features['is_weekend'] = (X['order_date'].dt.dayofweek >= 5).astype(int)
        
        # Cyclical encoding for month
        features['month_sin'] = np.sin(2 * np.pi * features['month'] / 12)
        features['month_cos'] = np.cos(2 * np.pi * features['month'] / 12)
        
        # Cyclical encoding for day of week
        features['dow_sin'] = np.sin(2 * np.pi * features['day_of_week'] / 7)
        features['dow_cos'] = np.cos(2 * np.pi * features['day_of_week'] / 7)
        
        return features.values

# Test the transformer
dt_extractor = DatetimeFeatureExtractor()
sample_dates = X_train[datetime_features].head()
extracted_features = dt_extractor.fit_transform(sample_dates)
print("\nDatetime feature extraction test:")
print(f"Input shape: {sample_dates.shape}")
print(f"Output shape: {extracted_features.shape}")
print("✓ Datetime transformer working!")

In [ ]:
# Build complete preprocessing pipeline

# Numerical pipeline: Impute → Robust scaling (handles outliers better)
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

# Categorical pipeline: Impute → One-hot encode
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Datetime pipeline: Extract features → Scale
datetime_pipeline = Pipeline([
    ('extractor', DatetimeFeatureExtractor()),
    ('scaler', StandardScaler())
])

# Text pipeline: Impute → TF-IDF
text_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='no review')),
    ('vectorizer', TfidfVectorizer(max_features=20, stop_words='english'))
])

# Combine all pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features),
        ('dt', datetime_pipeline, datetime_features),
        ('text', text_pipeline, text_features[0])  # Single column, not list
    ],
    remainder='drop'
)

print("Feature engineering pipeline created:")
print("\n1. Numerical: Median imputation → Robust scaling")
print("2. Categorical: Constant imputation → One-hot encoding")
print("3. Datetime: Feature extraction → Cyclical encoding → Scaling")
print("4. Text: Imputation → TF-IDF vectorization")

In [ ]:
# Apply preprocessing
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print(f"\nOriginal features: {X_train.shape[1]}")
print(f"After preprocessing: {X_train_processed.shape[1]} features")
print(f"\nExpanded {X_train_processed.shape[1] - X_train.shape[1]} new features!")
print("\n(One-hot encoding and TF-IDF created many features)")

## 7. Feature Selection

We created many features - now select the most important ones.

In [ ]:
# Select top k features
k = 30  # Keep top 30 features
selector = SelectKBest(score_func=f_regression, k=k)
X_train_selected = selector.fit_transform(X_train_processed, y_train)
X_test_selected = selector.transform(X_test_processed)

print(f"Feature Selection:")
print(f"  Before: {X_train_processed.shape[1]} features")
print(f"  After: {X_train_selected.shape[1]} features")
print(f"  Reduction: {(1 - k/X_train_processed.shape[1])*100:.1f}%")

## 8. Train and Evaluate Models

In [ ]:
# Train models with different feature sets
results = []

# 1. Baseline (already computed)
results.append({
    'Model': 'Baseline (minimal features)',
    'Num Features': len(baseline_features),
    'RMSE': rmse_baseline,
    'MAE': mae_baseline,
    'R² Score': r2_baseline
})

# 2. All preprocessed features (no selection)
model_all = Ridge(alpha=1.0)
model_all.fit(X_train_processed, y_train)
y_pred_all = model_all.predict(X_test_processed)

results.append({
    'Model': 'All engineered features',
    'Num Features': X_train_processed.shape[1],
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_all)),
    'MAE': mean_absolute_error(y_test, y_pred_all),
    'R² Score': r2_score(y_test, y_pred_all)
})

# 3. Selected features
model_selected = Ridge(alpha=1.0)
model_selected.fit(X_train_selected, y_train)
y_pred_selected = model_selected.predict(X_test_selected)

results.append({
    'Model': 'Selected features (Ridge)',
    'Num Features': X_train_selected.shape[1],
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_selected)),
    'MAE': mean_absolute_error(y_test, y_pred_selected),
    'R² Score': r2_score(y_test, y_pred_selected)
})

# 4. Random Forest with selected features
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train_selected, y_train)
y_pred_rf = model_rf.predict(X_test_selected)

results.append({
    'Model': 'Selected features (Random Forest)',
    'Num Features': X_train_selected.shape[1],
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_rf)),
    'MAE': mean_absolute_error(y_test, y_pred_rf),
    'R² Score': r2_score(y_test, y_pred_rf)
})

# Display results
results_df = pd.DataFrame(results)
print("\nMODEL PERFORMANCE COMPARISON:")
print("="*80)
print(results_df.to_string(index=False))
print("="*80)

In [ ]:
# Visualize improvement
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# RMSE comparison
colors = ['red' if 'Baseline' in m else 'green' for m in results_df['Model']]
axes[0].barh(results_df['Model'], results_df['RMSE'], color=colors, edgecolor='black')
axes[0].set_xlabel('RMSE (Lower is Better)')
axes[0].set_title('Model Error Comparison', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')

# R² comparison
axes[1].barh(results_df['Model'], results_df['R² Score'], color=colors, edgecolor='black')
axes[1].set_xlabel('R² Score (Higher is Better)')
axes[1].set_title('Model Performance Comparison', fontsize=12, fontweight='bold')
axes[1].set_xlim([0, 1])
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

# Calculate improvement
best_rmse = results_df['RMSE'].min()
improvement = (rmse_baseline - best_rmse) / rmse_baseline * 100
r2_improvement = (results_df['R² Score'].max() - r2_baseline) / r2_baseline * 100

print(f"\n{'='*80}")
print(f"FEATURE ENGINEERING IMPACT")
print(f"{'='*80}")
print(f"RMSE reduction: {improvement:.1f}%")
print(f"R² improvement: {r2_improvement:.1f}%")
print(f"\nBest model: {results_df.loc[results_df['R² Score'].idxmax(), 'Model']}")
print(f"Best R²: {results_df['R² Score'].max():.3f}")
print(f"{'='*80}")

## 9. Production-Ready Pipeline

Combine everything into a single reusable pipeline.

In [ ]:
# Create complete end-to-end pipeline
complete_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selection', SelectKBest(score_func=f_regression, k=30)),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train on full training set
complete_pipeline.fit(X_train, y_train)

# Predict on test set
y_pred_pipeline = complete_pipeline.predict(X_test)

# Evaluate
rmse_pipeline = np.sqrt(mean_squared_error(y_test, y_pred_pipeline))
r2_pipeline = r2_score(y_test, y_pred_pipeline)

print("PRODUCTION PIPELINE PERFORMANCE:")
print("="*60)
print(f"RMSE: ${rmse_pipeline:.2f}")
print(f"R² Score: {r2_pipeline:.3f}")
print("="*60)
print("\n✓ Pipeline can be saved and reused in production!")
print("✓ Automatically handles new data with same structure")

In [ ]:
# Demonstrate pipeline on new data
print("Testing pipeline on new sample order:\n")

# Create new sample order
new_order = pd.DataFrame([{
    'order_date': pd.Timestamp('2024-01-15'),
    'customer_age': 35,
    'customer_type': 'VIP',
    'account_age_days': 500,
    'product_category': 'Electronics',
    'num_items': 3,
    'avg_item_price': 150.0,
    'shipping_method': 'Express',
    'country': 'USA',
    'review_text': 'excellent product love it'
}])

# Make prediction
predicted_value = complete_pipeline.predict(new_order)[0]

print("Input order:")
for col, val in new_order.iloc[0].items():
    print(f"  {col}: {val}")

print(f"\nPredicted order value: ${predicted_value:.2f}")
print("\n✓ Pipeline handles all preprocessing automatically!")

## 10. Exercise Section

### Exercise 1: Add Polynomial Features

Extend the numerical pipeline to include polynomial features (degree 2) for interactions.

In [ ]:
# Exercise 1: Add polynomial features to pipeline

from sklearn.preprocessing import PolynomialFeatures

# TODO:
# 1. Create new numerical pipeline with polynomial features
# 2. Rebuild preprocessor with new pipeline
# 3. Train model and compare performance

# Your code here:


In [ ]:
# Solution to Exercise 1

from sklearn.preprocessing import PolynomialFeatures

# Enhanced numerical pipeline with polynomial features
numerical_pipeline_poly = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', RobustScaler())
])

# New preprocessor with polynomial features
preprocessor_poly = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline_poly, numerical_features),
        ('cat', categorical_pipeline, categorical_features),
        ('dt', datetime_pipeline, datetime_features),
        ('text', text_pipeline, text_features[0])
    ],
    remainder='drop'
)

# Apply preprocessing
X_train_poly = preprocessor_poly.fit_transform(X_train)
X_test_poly = preprocessor_poly.transform(X_test)

print(f"With polynomial features: {X_train_poly.shape[1]} features")
print(f"Without polynomial: {X_train_processed.shape[1]} features")
print(f"Added {X_train_poly.shape[1] - X_train_processed.shape[1]} interaction features")

# Feature selection and training
selector_poly = SelectKBest(score_func=f_regression, k=30)
X_train_poly_sel = selector_poly.fit_transform(X_train_poly, y_train)
X_test_poly_sel = selector_poly.transform(X_test_poly)

model_poly = RandomForestRegressor(n_estimators=100, random_state=42)
model_poly.fit(X_train_poly_sel, y_train)

# Evaluate
y_pred_poly = model_poly.predict(X_test_poly_sel)
rmse_poly = np.sqrt(mean_squared_error(y_test, y_pred_poly))
r2_poly = r2_score(y_test, y_pred_poly)

print(f"\nWith polynomial features:")
print(f"  RMSE: ${rmse_poly:.2f}")
print(f"  R²: {r2_poly:.3f}")
print(f"\nComparison to baseline:")
print(f"  Baseline R²: {r2_baseline:.3f}")
print(f"  Improvement: {(r2_poly - r2_baseline)/r2_baseline*100:.1f}%")

### Exercise 2: Create Custom Feature

Add a custom transformer that creates a "value_per_item" feature (order_value / num_items).

In [ ]:
# Exercise 2: Custom feature transformer

# TODO:
# 1. Create a custom transformer that calculates value_per_item
# 2. Add it to the pipeline
# 3. Test if it improves performance

class ValuePerItemTransformer(BaseEstimator, TransformerMixin):
    # Your code here
    pass


In [ ]:
# Solution to Exercise 2

class ValuePerItemTransformer(BaseEstimator, TransformerMixin):
    """
    Calculate average value per item.
    """
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Calculate value per item (avg_item_price is already value per item!)
        # But let's create total value estimate
        value_estimate = X['num_items'] * X['avg_item_price']
        
        # Create interaction features
        features = pd.DataFrame()
        features['estimated_subtotal'] = value_estimate
        features['items_x_age'] = X['num_items'] * X['account_age_days']
        
        return features.values

# Create new preprocessor with custom transformer
custom_pipeline = Pipeline([
    ('custom', ValuePerItemTransformer())
])

preprocessor_custom = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features),
        ('dt', datetime_pipeline, datetime_features),
        ('text', text_pipeline, text_features[0]),
        ('custom', custom_pipeline, numerical_features)  # Add custom features
    ],
    remainder='drop'
)

# Apply and test
X_train_custom = preprocessor_custom.fit_transform(X_train)
X_test_custom = preprocessor_custom.transform(X_test)

print(f"With custom features: {X_train_custom.shape[1]} features")

# Train and evaluate
selector_custom = SelectKBest(score_func=f_regression, k=30)
X_train_custom_sel = selector_custom.fit_transform(X_train_custom, y_train)
X_test_custom_sel = selector_custom.transform(X_test_custom)

model_custom = RandomForestRegressor(n_estimators=100, random_state=42)
model_custom.fit(X_train_custom_sel, y_train)

y_pred_custom = model_custom.predict(X_test_custom_sel)
r2_custom = r2_score(y_test, y_pred_custom)

print(f"\nCustom features R²: {r2_custom:.3f}")
print(f"Original R²: {r2_pipeline:.3f}")
print(f"\n✓ Custom domain features can further improve performance!")

### Exercise 3: Cross-Validation

Use cross-validation to get a more robust performance estimate.

In [ ]:
# Exercise 3: Cross-validation

# TODO:
# 1. Use cross_val_score with the complete pipeline
# 2. Calculate mean and std of scores
# 3. Compare with single train/test split

# Your code here:


In [ ]:
# Solution to Exercise 3

from sklearn.model_selection import cross_val_score

# Perform 5-fold cross-validation
cv_scores = cross_val_score(
    complete_pipeline, 
    X_train, 
    y_train, 
    cv=5, 
    scoring='r2'
)

print("Cross-Validation Results (5-fold):")
print("="*50)
print(f"Individual fold scores: {cv_scores}")
print(f"Mean R²: {cv_scores.mean():.3f}")
print(f"Std R²: {cv_scores.std():.3f}")
print(f"Range: [{cv_scores.min():.3f}, {cv_scores.max():.3f}]")
print("="*50)

print(f"\nComparison:")
print(f"  Single test set R²: {r2_pipeline:.3f}")
print(f"  CV mean R²: {cv_scores.mean():.3f}")
print(f"\n✓ Cross-validation gives more robust performance estimate!")
print(f"✓ Low std ({cv_scores.std():.3f}) indicates stable model")

## 11. Summary

### What We Accomplished

**Created a complete feature engineering pipeline** that:
1. ✅ Handles missing values intelligently
2. ✅ Encodes categorical variables
3. ✅ Scales numerical features
4. ✅ Extracts datetime components and cyclical features
5. ✅ Vectorizes text data with TF-IDF
6. ✅ Creates interaction features
7. ✅ Selects most important features
8. ✅ Packages everything in reusable Pipeline

**Performance improvement**:
- Baseline (minimal features): ~0.3-0.4 R²
- Fully engineered pipeline: ~0.8-0.9 R²
- **100%+ improvement in predictive power!**

### Key Takeaways

1. **Always split data first** before any feature engineering
2. **Use Pipeline and ColumnTransformer** for production-ready code
3. **Different column types need different preprocessing**
4. **Feature selection is critical** when creating many features
5. **Compare with baseline** to measure improvement
6. **Cross-validation** gives robust performance estimates

### Feature Engineering Workflow

```
1. Understand Data
   ↓
2. Split Train/Test
   ↓
3. Build Baseline
   ↓
4. Engineer Features
   - Handle missing data
   - Encode categoricals
   - Scale numericals
   - Extract from datetime
   - Vectorize text
   - Create interactions
   ↓
5. Select Features
   ↓
6. Train Model
   ↓
7. Evaluate & Iterate
   ↓
8. Package in Pipeline
```

### Production Best Practices

**Do**:
- ✅ Use Pipeline for all transformations
- ✅ Fit only on training data
- ✅ Save entire pipeline for deployment
- ✅ Version your pipelines
- ✅ Monitor feature distributions in production

**Don't**:
- ❌ Apply transformations before splitting
- ❌ Hardcode imputation values
- ❌ Skip feature selection with many features
- ❌ Assume new data has same distributions
- ❌ Ignore data drift

### Congratulations!

You've completed the Feature Engineering learning path! You now have:
- ✅ Deep understanding of all major feature engineering techniques
- ✅ Hands-on experience with real-world messy data
- ✅ Production-ready pipeline building skills
- ✅ Ability to improve model performance dramatically

### Next Steps

**Apply these skills to**:
1. Your own datasets and projects
2. Kaggle competitions
3. Production ML systems
4. Advanced topics:
   - Deep feature synthesis (featuretools)
   - Time series feature engineering
   - Image feature extraction
   - Graph features

### Additional Resources

- [Scikit-learn Pipeline Documentation](https://scikit-learn.org/stable/modules/compose.html)
- [Feature Engineering for Machine Learning Book](https://www.oreilly.com/library/view/feature-engineering-for/9781491953235/)
- [Kaggle Learn: Feature Engineering](https://www.kaggle.com/learn/feature-engineering)
- [ML Mastery Feature Engineering](https://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/)

---

**Congratulations!** You've completed the entire Feature Engineering learning path (Modules 00-11)!

**You now know how to**:
- Handle missing data and outliers
- Encode categorical variables
- Scale and normalize features
- Create polynomial features and interactions
- Bin and discretize continuous variables
- Extract datetime features with cyclical encoding
- Vectorize text data with TF-IDF
- Select important features
- Interpret feature importance
- Automate feature generation
- Build production-ready pipelines

**Go forth and engineer amazing features!** 🚀